In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !wget http://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip   

In [ ]:
# !unzip social_honeypot_icwsm_2011.zip

In [ ]:
import pandas as pd
import numpy as np
import re, string
import operator
import networkx as nx
from numba import jit, cuda

import timeit

In [ ]:
!pip install scikit-plot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from operator import mul
import functools
from community import community_louvain
import community
import networkx as nx
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from scipy.stats import zscore

import matplotlib.pyplot as plt
from operator import itemgetter

In [ ]:

def func(string):
   return pd.Series(string.split(',')).astype(int)
  
  
dfContent_polluters = pd.read_csv('social_honeypot_icwsm_2011/content_polluters.txt',sep='\t', names=
                 ["UserID","CreatedAt","CollectedAt","NumberOfFollowings","NumberOfFollowers",
                  "NumberOfTweets","LengthOfScreenName","LengthOfDescriptionInUserProfile"],
                parse_dates=["CreatedAt","CollectedAt"])

dfContent_polluters_followings = pd.read_csv('social_honeypot_icwsm_2011/content_polluters_followings.txt',sep='\t', names=
                 ["UserID","SeriesOfNumberOfFollowings"],converters={"SeriesOfNumberOfFollowings":func},index_col=0)

dfContent_polluters_tweets = pd.read_csv('social_honeypot_icwsm_2011/content_polluters_tweets.txt',sep='\t', names=
                 ["UserID","TweetID","Tweet","CreatedAt"], parse_dates=["CreatedAt"],index_col=0)

dfLegitimate_users = pd.read_csv('social_honeypot_icwsm_2011/legitimate_users.txt',sep='\t', names=
                 ["UserID","CreatedAt","CollectedAt","NumberOfFollowings","NumberOfFollowers",
                  "NumberOfTweets","LengthOfScreenName","LengthOfDescriptionInUserProfile"],
                parse_dates=["CreatedAt","CollectedAt"])

dfLegitimate_users_followings = pd.read_csv('social_honeypot_icwsm_2011/legitimate_users_followings.txt',sep='\t', names=
                 ["UserID","SeriesOfNumberOfFollowings"],converters={"SeriesOfNumberOfFollowings":func},index_col=0)


dfLegitimate_users_tweets = pd.read_csv('social_honeypot_icwsm_2011/legitimate_users_tweets.txt',sep='\t', names=
                 ["UserID","TweetID","Tweet","CreatedAt"], parse_dates=["CreatedAt"],index_col=0)

In [ ]:
start = timeit.default_timer()

In [ ]:
otherFakeUser = []
with open('/content/drive/MyDrive/otherfakeuser.txt', 'r') as f:
    for line in f:
        otherFakeUser.append(line)

In [ ]:
len(otherFakeUser)

18179

In [ ]:
def get_user_id(data_frame):
    df = pd.DataFrame({"UserID": data_frame["UserID"]})
    return df

In [ ]:
data_Users_polluters = get_user_id(dfContent_polluters)
data_Users_legitimate = get_user_id(dfLegitimate_users)

# data_Users_polluters.insert(0,"Class",1)
# data_Users_legitimate.insert(0,"Class",0)

data_set_users = pd.concat([data_Users_polluters, data_Users_legitimate])

print("data_set_users : ",len(data_set_users))

    # /////////////////////////////////////////////////////
    # print("dfLegitimate_users_followings : ",len(dfLegitimate_users_followings))
    # print("dfContent_polluters_followings : ",len(dfContent_polluters_followings))

# dfLegitimate_users_followings = dfLegitimate_users_followings.head(100)
# dfContent_polluters_followings = dfContent_polluters_followings.head(100)

data_set_users :  41499


In [ ]:

# /////////////////////////////////////////////////////

node_list = []

for UserID, SeriesOfNumberOfFollowings in dfLegitimate_users_followings.itertuples():
    node_list.append(int(UserID))
    for fo in SeriesOfNumberOfFollowings:
        # if fo in data_set_users['UserID'].values :
            node_list.append(int(fo))

for UserID, SeriesOfNumberOfFollowings in dfContent_polluters_followings.itertuples():
    node_list.append(int(UserID))
    for fo in SeriesOfNumberOfFollowings:
        # if fo in data_set_users['UserID'].values :
            node_list.append(int(fo))

node_list = list(set(node_list))
print("node_list : ",len(node_list))

    # dfUsersNode = pd.DataFrame(node_list)
    # export_csv = dfUsersNode.to_csv (r'C:/Users/Nill/Desktop/FALL1400/social_honeypot_icwsm_2011/dfUsersNode.csv', index = None, header=True)
    # ///////////////////////////////////////////////////////



node_list :  93247


In [ ]:
import random
node_list_fake = random.sample(node_list, 30000)

print("node_list_fake : ",len(node_list_fake))


node_list_fake :  30000


In [ ]:
edge_list = []

for UserID, SeriesOfNumberOfFollowings in dfLegitimate_users_followings.itertuples():
    for fo in SeriesOfNumberOfFollowings:
        # if fo in data_set_users['UserID'].values :
            edge_list.append((int(UserID), int(fo)))

for UserID, SeriesOfNumberOfFollowings in dfContent_polluters_followings.itertuples():
    for fo in SeriesOfNumberOfFollowings:
        # if fo in data_set_users['UserID'].values :
            edge_list.append((int(UserID), int(fo)))

print("edge_list : ", len(edge_list))

edge_list :  8950572


In [ ]:
G = nx.Graph()
G.add_nodes_from(node_list)
G.add_edges_from(edge_list)

In [ ]:
print("The graph has", len(G.nodes), "nodes and", len(G.edges), "edges")

The graph has 93247 nodes and 1416359 edges


In [ ]:
node_and_degree = G.degree()
(largest_hub, degree) = sorted(node_and_degree, key=itemgetter(1))[-1]
print(largest_hub,degree)

2001 3981


In [ ]:
# G_new=G.to_undirected()
G_new=G
partition = community_louvain.best_partition(G_new)

In [ ]:
dendo = community_louvain.generate_dendrogram(G_new)
for level in range(0,len(dendo) - 1):
    print("partition at level", level,
          "is", community_louvain.partition_at_level(dendo, level))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
community_louvain.modularity(partition, G_new)

0.7072375595457567

In [ ]:
dictionary=dict()

In [ ]:
# Keep only communities with a minimum of authors
counter=0
centers = {}
communities = {}
fake_communities = []
fake_communities_num = 0
allNodes = []
G_main_com = G_new.copy()
min_nb = 2
for com in set(partition.values()) :
    list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com]
    if len(list_nodes) < min_nb:
        G_main_com.remove_nodes_from(list_nodes)
    else:
        # Get center
        H = G_main_com.subgraph(list_nodes)
        d_c = nx.degree_centrality(H)
        center = max(d_c, key=d_c.get)
        allNodes.append(center)
        centers[center] = com
        communities[com] = center
        # Print community
        fakeCounter = 0

        for userNode in list_nodes:
            allNodes.append(userNode)
            if userNode in data_Users_polluters['UserID'].values : 
                fakeCounter = fakeCounter + 1
            elif userNode in otherFakeUser :
                fakeCounter = fakeCounter + 1
            elif userNode in node_list_fake :
                fakeCounter = fakeCounter + 1
        centerType = ""        
        if center in data_Users_polluters['UserID'].values : 
           centerType = "Fake"
          #  communities[1] = center
        elif center in otherFakeUser : 
           centerType = "Fake"
          #  communities[1] = center
        else :
        # center in data_Users_legitimate['UserID'].values : 
           centerType = "Real"
          #  communities[0] = center
        # else : 
        #   centerType = "Unknown"
        if fakeCounter/len(list_nodes) > 0.65 :
          fake_communities.append(com)
          fake_communities_num = fake_communities_num+1
        print('Community of ', center ,'  :  ' , centerType , '(ID ', com, ') - countOfMember : ', len(list_nodes) , ' - countOfFakeMember : ' , fakeCounter , ' members:' , fakeCounter/len(list_nodes))
        counter=counter+1
        # print(list_nodes, '\n')
        dictionary[f'Community of ', center , '(ID ', com, ') - countOfMember : ', len(list_nodes) , ' - countOfFakeMember : ' , fakeCounter , '  fakeCounter/len(list_nodes)  ' , fakeCounter/len(list_nodes) ,' members:']=np.array(list_nodes)
print(fake_communities, '\n')

Community of  0   :   Real (ID  0 ) - countOfMember :  914  - countOfFakeMember :  354  members: 0.387308533916849
Community of  94   :   Real (ID  1 ) - countOfMember :  14870  - countOfFakeMember :  6586  members: 0.44290517821116343
Community of  244   :   Real (ID  2 ) - countOfMember :  7551  - countOfFakeMember :  4949  members: 0.6554098794861608
Community of  662   :   Real (ID  3 ) - countOfMember :  8546  - countOfFakeMember :  6817  members: 0.7976831266089398
Community of  2001   :   Real (ID  4 ) - countOfMember :  6715  - countOfFakeMember :  5910  members: 0.8801191362620998
Community of  107365486   :   Fake (ID  5 ) - countOfMember :  2461  - countOfFakeMember :  1586  members: 0.6444534741974807
Community of  84371497   :   Fake (ID  6 ) - countOfMember :  4994  - countOfFakeMember :  2691  members: 0.538846615939127
Community of  105506414   :   Fake (ID  7 ) - countOfMember :  6207  - countOfFakeMember :  2478  members: 0.39922667955534075
Community of  14516920   :

In [ ]:
allNodes = set(allNodes)

In [ ]:
len(allNodes)

93247

In [ ]:
subG=G.subgraph(G_main_com)
pos=nx.spring_layout(subG)

In [ ]:
nx.draw_networkx_nodes(subG,pos,nodelist=fake_communities,node_color='r',node_size=50, label='fake accounts')
nx.draw_networkx_nodes(subG,pos,nodelist=G_main_com,node_color='b',node_size=50, label='real accounts')
axes=plt.gca()
axes.set_ylim([-0.15,0.15])
plt.legend()
plt.show()

In [ ]:

with open('communityallNotExist.txt', 'w') as f:
    print(dictionary, file=f)

In [ ]:
nx.draw(fake_communities, pos=nx.spring_layout(fake_communities,k=.12),node_color='c',edge_color='k')

In [ ]:

nx.draw(G, pos=nx.spring_layout(G,k=.12),node_color='c',edge_color='k')


Community of fakeusers



In [ ]:

# # /////////////////////////////////////////////////////

# node_list_fakes = []


# for UserID, SeriesOfNumberOfFollowings in dfContent_polluters_followings.itertuples():
#     node_list_fakes.append(int(UserID))
#     for fo in SeriesOfNumberOfFollowings:
#         if fo in data_Users_polluters['UserID'].values :
#             node_list_fakes.append(int(fo))
#         elif fo in otherFakeUser:
#             node_list_fakes.append(int(fo))

# node_list_fakes = list(set(node_list_fakes))
# print("node_list : ",len(node_list_fakes))

#     # dfUsersNode = pd.DataFrame(node_list)
#     # export_csv = dfUsersNode.to_csv (r'C:/Users/Nill/Desktop/FALL1400/social_honeypot_icwsm_2011/dfUsersNode.csv', index = None, header=True)
#     # ///////////////////////////////////////////////////////



In [ ]:
# edge_list_fake = []

# for UserID, SeriesOfNumberOfFollowings in dfContent_polluters_followings.itertuples():
#     for fo in SeriesOfNumberOfFollowings:
#         if fo in data_Users_polluters['UserID'].values :
#             edge_list_fake.append((int(UserID), int(fo)))
#         elif fo in otherFakeUser:
#             edge_list_fake.append((int(UserID), int(fo)))

# print("edge_list : ", len(edge_list_fake))

In [ ]:
# G_fakes= nx.Graph()
# G_fakes.add_nodes_from(node_list_fakes)
# G_fakes.add_edges_from(edge_list_fake)

In [ ]:
# print("The graph has", len(G_fakes.nodes), "nodes and", len(G_fakes.edges), "edges")

In [ ]:
# node_and_degree_fakes = G_fakes.degree()
# (largest_hub_fakes, degree_fakes) = sorted(node_and_degree_fakes, key=itemgetter(1))[-1]
# print(largest_hub_fakes,degree_fakes)

In [ ]:
# # G_new=G.to_undirected()
# G_new_fakes=G_fakes
# partition_fakes = community_louvain.best_partition(G_new_fakes)

In [ ]:
# dendo_fakes = community_louvain.generate_dendrogram(G_new_fakes)
# for level in range(0,len(dendo_fakes) - 1):
#     print("partition at level _fakes " , level,
#           "is", community_louvain.partition_at_level(dendo_fakes, level))

In [ ]:
# community_louvain.modularity(partition_fakes, G_new_fakes)

In [ ]:
# dictionary_fakes=dict()

In [ ]:
# # Keep only communities with a minimum of authors
# counter_fakes=0
# centers_fakes = {}
# communities_fakes = {}
# fake_communities_fakes = []
# fake_communities_num_fakes = 0
# allNodes_fakes = []
# G_main_com_fakes = G_new_fakes.copy()
# min_nb_fakes = 2
# for com in set(partition_fakes.values()) :
#     list_nodes_fakes = [nodes for nodes in partition_fakes.keys() if partition_fakes[nodes] == com]
#     if len(list_nodes_fakes) < min_nb_fakes:
#         G_main_com_fakes.remove_nodes_from(list_nodes_fakes)
#     else:
#         # Get center
#         H = G_main_com_fakes.subgraph(list_nodes_fakes)
#         d_c = nx.degree_centrality(H)
#         center_fakes = max(d_c, key=d_c.get)
#         allNodes_fakes.append(center_fakes)
#         centers_fakes[center_fakes] = com
#         communities_fakes[com] = center_fakes
#         # Print community
#         fakeCounter_fakes = 0

#         for userNode in list_nodes_fakes:
#             allNodes_fakes.append(userNode)
#             if userNode in data_Users_polluters['UserID'].values : 
#                 fakeCounter_fakes = fakeCounter_fakes + 1
#             elif userNode in otherFakeUser :
#                 fakeCounter_fakes = fakeCounter_fakes + 1
#         centerType = ""        
#         if center_fakes in data_Users_polluters['UserID'].values : 
#            centerType = "Fake"
#           #  communities[1] = center
#         elif center in otherFakeUser : 
#            centerType = "Fake"
#           #  communities[1] = center
#         else :
#         # center in data_Users_legitimate['UserID'].values : 
#            centerType = "Real"
#           #  communities[0] = center
#         # else : 
#         #   centerType = "Unknown"
#         if fakeCounter_fakes/len(list_nodes_fakes) > 0.65 :
#           fake_communities_fakes.append(com)
#           fake_communities_num_fakes = fake_communities_num_fakes+1
#         print('Community of ', center_fakes ,'  :  ' , centerType , '(ID ', com, ') - countOfMember : ', len(list_nodes_fakes) , ' - countOfFakeMember : ' , fakeCounter_fakes , ' members:' , fakeCounter_fakes/len(list_nodes_fakes))
#         counter=counter+1
#         # print(list_nodes, '\n')
#         dictionary_fakes[f'Community of ', center_fakes , '(ID ', com, ') - countOfMember : ', len(list_nodes_fakes) , ' - countOfFakeMember : ' , fakeCounter_fakes , '  fakeCounter_fakes/len(list_nodes_fakes)  ' , fakeCounter_fakes/len(list_nodes_fakes)  ,  ' members:']=np.array(list_nodes_fakes)
# print(fake_communities_fakes, '\n')

In [ ]:
# # 
# with open('communityallFakes.txt', 'w') as f:
#     print(dictionary_fakes, file=f)

## classification


In [ ]:
# import tweepy as tweepy

In [ ]:
# # consumer_key = 'Cq1sycJH2dda1jZCvMlqiBT7b'
# # consumer_secret = 'NAmkgtvnezW42U80ndOqeD8bfIZnM8KF8k9g9wl73SNa9mjfis'
# # access_token = '1544279124482703364-Taf0ZjnGuRW4HCfAidh4jhRK4xZ3RI'
# # access_token_secret = 'xtst9BPdI4AGs1itwmmXSII5J8V25D2vyRj7lKYdnqHKJ'

# consumer_key = 'BJZXO2O7qffWEWiBzztaI0pmz'
# consumer_secret = '1V78UxJL3ahxfeyRxchZX9OD5Se35NP0FoRVk2Kt1Q6arV1s9t'
# access_token = '1421415547607126016-Gt5QxcyBoPTQr1rKRRogxhgO46JdrW'
# access_token_secret = 'afPcI7OlTv9Vbm55ecSijEWhRIDbhPdfOnz2lnGF8qIal'

# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)

# api = tweepy.API(auth , wait_on_rate_limit=True)


In [ ]:
# with open('users_all_Get.txt', 'w') as f:
#     for line in allNodes:
#         f.write(f"{line}\n")

In [ ]:
# def func(string):
#    return pd.Series(string.split(',')).astype(int)
  

In [ ]:

# def folowersSTD(user) :
#     freiend_list = ''
#     friends = api.friends(id=user)[:10]
#     # print(len(friends))
#     for friend in friends:
#         freiend_list = freiend_list+str(friend.id)+','
#     # print(func(freiend_list[:-1]))
#     vari = func(freiend_list[:-1])
#     return vari.std()

In [ ]:
# from datetime import datetime
# now = datetime.now()
 
# print("now =", now)
# dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
# print("date and time =", dt_string)	


In [ ]:
# def getTweets(user) :
#     tweets = api.user_timeline(id=user, 
#                            # 200 is the maximum allowed count
#                            count=10,
#                            include_rts = False,
#                            # Necessary to keep full_text 
#                            # otherwise only the first 140 words are extracted
#                            tweet_mode = 'extended'
#                            )
#     return tweets

In [ ]:
# import time

In [ ]:
# f_account10000 = open('/content/drive/MyDrive/fake_users_account10000.txt', 'w') 

In [ ]:
# from itertools import islice

In [ ]:
# notExistUser10000 = []
# counter = 0
# iterator = islice(allNodes, 10000)
# # for seed_id , SeriesOfNumberOfFollowings in dfContent_polluters_followings[:10].itertuples():
# #     print("***************** seed_id ", seed_id)
# for userId in iterator :
#   counter = counter + 1
#   try:
#       friend=api.get_user(userId)
#       # f_account.write(str(friend.id))
#       # f_account.write('\t')
#       # f_account.write(str(friend.created_at))
#       # f_account.write('\t')
#       # f_account.write(str(now))
#       # f_account.write('\t')
#       # f_account.write(str(friend.friends_count))
#       # f_account.write('\t')
#       # f_account.write(str(friend.followers_count))
#       # f_account.write('\t')
#       # f_account.write(str(friend.statuses_count))
#       # f_account.write('\t')
#       # f_account.write(str(folowersSTD(friend.id)))
#       # f_account.write('\t')
#   except Exception as e: 
#         print(e , "   " , counter ) 
#         notExistUser10000.append(userId)
#         f_account10000.write(str(userId))
#         f_account10000.write('\n')
#         pass

In [ ]:
# f_account10000.close()

In [ ]:
# f_account20000 = open('/content/drive/MyDrive/fake_users_account20000.txt', 'w') 

In [ ]:
# notExistUser20000 = []
# counter = 0
# iterator = islice(allNodes, 10000,20000)
# # for seed_id , SeriesOfNumberOfFollowings in dfContent_polluters_followings[:10].itertuples():
# #     print("***************** seed_id ", seed_id)
# for userId in iterator :
#   counter = counter + 1
#   try:
#       friend=api.get_user(userId)
#       # f_account.write(str(friend.id))
#       # f_account.write('\t')
#       # f_account.write(str(friend.created_at))
#       # f_account.write('\t')
#       # f_account.write(str(now))
#       # f_account.write('\t')
#       # f_account.write(str(friend.friends_count))
#       # f_account.write('\t')
#       # f_account.write(str(friend.followers_count))
#       # f_account.write('\t')
#       # f_account.write(str(friend.statuses_count))
#       # f_account.write('\t')
#       # f_account.write(str(folowersSTD(friend.id)))
#       # f_account.write('\t')
#   except Exception as e: 
#         print(e , "   " , counter ) 
#         notExistUser20000.append(userId)
#         f_account20000.write(str(userId))
#         f_account20000.write('\n')
#         pass

In [ ]:
# f_account20000.close()

# classification done

# plot

In [ ]:

# with open('communityallNotExist.txt', 'w') as f:
#     print(dictionary, file=f)
    

In [ ]:
print(counter)

In [ ]:
stop = timeit.default_timer()
print('Time: ', stop - start)  

In [ ]:
import seaborn as sns

In [ ]:
# # Display graph
# plt.figure(figsize=(20, 10))
# node_size = 50
# count = 0
# pos = nx.spring_layout(G_main_com)
# colors = dict(zip(communities.keys(), sns.color_palette('hls', len(communities.keys()))))

# for com in communities.keys():
#     count = count  +1
#     list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com and nodes not in communities.values()]
#     nx.draw_networkx_nodes(G_main_com, pos, list_nodes, node_size = node_size, node_color = colors[com])
#     nx.draw_networkx_nodes(G_main_com, pos, list([communities[com]]), node_size = node_size*5, node_color = colors[com])
# nx.draw_networkx_edges(G_main_com, pos, alpha=0.5)
# # labels = {k: str(v) + ': ' + str(k) for k,v in centers.items()}
# labels = {k: str(v) for k,v in centers.items()}
# nx.draw_networkx_labels(G_main_com, pos, labels)
# plt.axis('off')
# plt.show()

In [ ]:
# Display graph
plt.figure(figsize=(100, 40))
node_size = 50
count = 0
pos = nx.spring_layout(G_main_com)
colors = dict(zip(communities.keys(), sns.color_palette('hls', len(communities.keys()))))

for com in communities.keys():
    count = count  +1
    list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com and nodes not in communities.values()]
    nx.draw_networkx_nodes(G_main_com, pos, list_nodes, node_size = node_size, node_color = colors[com])
    nx.draw_networkx_nodes(G_main_com, pos, list([communities[com]]), node_size = node_size*5, node_color = colors[com])
nx.draw_networkx_edges(G_main_com, pos, alpha=0.5)
# labels = {k: str(v) + ': ' + str(k) for k,v in centers.items()}
labels = {k: str(v) for k,v in centers.items()}
nx.draw_networkx_labels(G_main_com, pos, labels)
plt.axis('off')
plt.show()

In [ ]:
# # Display graph
# plt.figure(figsize=(200, 80),dpi=100)
# node_size = 50
# count = 0
# pos = nx.spring_layout(G_main_com)
# colors = dict(zip(communities.keys(), sns.color_palette('hls', len(communities.keys()))))

# for com in communities.keys():
#     count = count  +1
#     list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com and nodes not in communities.values()]
#     nx.draw_networkx_nodes(G_main_com, pos, list_nodes, node_size = node_size, node_color = colors[com])
#     nx.draw_networkx_nodes(G_main_com, pos, list([communities[com]]), node_size = node_size*5, node_color = colors[com])
# nx.draw_networkx_edges(G_main_com, pos, alpha=0.5)
# # labels = {k: str(v) + ': ' + str(k) for k,v in centers.items()}
# labels = {k: str(v) for k,v in centers.items()}
# nx.draw_networkx_labels(G_main_com, pos, labels)
# plt.axis('off')
# plt.show()

In [ ]:
# # Display graph allnotexist
# plt.figure(figsize=(40, 20))
# node_size = 50
# count = 0
# pos = nx.spring_layout(G_main_com)
# colors = dict(zip(communities.keys(), sns.color_palette('hls', len(communities.keys()))))

# for com in communities.keys():
#     count = count  +1
#     list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com and nodes not in communities.values()]
#     nx.draw_networkx_nodes(G_main_com, pos, list_nodes, node_size = node_size, node_color = colors[com])
#     nx.draw_networkx_nodes(G_main_com, pos, list([communities[com]]), node_size = node_size*5, node_color = colors[com])
# nx.draw_networkx_edges(G_main_com, pos, alpha=0.5)
# # labels = {k: str(v) + ': ' + str(k) for k,v in centers.items()}
# labels = {k: str(v) for k,v in centers.items()}
# nx.draw_networkx_labels(G_main_com, pos, labels)
# plt.axis('off')
# plt.show()

In [ ]:
# # Display graph all fakes
# plt.figure(figsize=(10, 5))
# node_size = 50
# count = 0
# pos = nx.spring_layout(G_main_com_fakes)
# colors = dict(zip(communities_fakes.keys(), sns.color_palette('hls', len(communities_fakes.keys()))))

# for com in communities_fakes.keys():
#     count = count  +1
#     list_nodes = [nodes for nodes in partition_fakes.keys() if partition_fakes[nodes] == com and nodes not in communities_fakes.values()]
#     nx.draw_networkx_nodes(G_main_com_fakes, pos, list_nodes, node_size = node_size, node_color = colors[com])
#     nx.draw_networkx_nodes(G_main_com_fakes, pos, list([communities_fakes[com]]), node_size = node_size*5, node_color = colors[com])
# nx.draw_networkx_edges(G_main_com_fakes, pos, alpha=0.5)
# # labels = {k: str(v) + ': ' + str(k) for k,v in centers.items()}
# labels = {k: str(v) for k,v in centers_fakes.items()}
# nx.draw_networkx_labels(G_main_com_fakes, pos, labels)
# plt.axis('off')
# plt.show()

In [ ]:
# # Display graph
# plt.figure(figsize=(50, 40))
# node_size = 50
# count = 0
# pos = nx.spring_layout(G_main_com)
# colors = dict(zip(communities.keys(), sns.color_palette('hls', len(communities.keys()))))


# labels = {}    
# for node in G_main_com.nodes():
#     if node in data_Users_polluters :
#         #set the node name as the key and the label as its value 
#         labels[node] = "Fake"

#     else :
#         labels[node] = "Real"
# #set the argument 'with labels' to False so you have unlabeled graph

# for com in communities.keys():
#     count = count  +1
#     list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com and nodes not in communities.values()]
#     nx.draw_networkx_nodes(G_main_com, pos, list_nodes, node_size = node_size, node_color =colors[com])
#     nx.draw_networkx_nodes(G_main_com, pos, list([communities[com]]), node_size = node_size*5, node_color = colors[com])
# nx.draw_networkx_edges(G_main_com, pos, alpha=0.5)
# # labels = {k: str(v) + ': ' + str(k) for k,v in centers.items()}
# nx.draw_networkx_labels(G_main_com, pos, labels)
# plt.axis('off')
# plt.show()

In [ ]:
stop = timeit.default_timer()
print('Time: ', stop - start)  